In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

### Extract Data

In [2]:
#Get Settlement List
Settlements = pd.read_csv("afg_ppl_settlement_pnt.csv",sep=",")
Settlements = Settlements[[0,10,11]]
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

#Get District Court List
Districts = pd.read_csv("District_Courts.csv",sep=",")
Districts = Districts[[1,4,5]]
D = Districts.shape[0]
District_List = Districts['DIST_CODE'].tolist()

#Get Appeals Court List
Appeals = pd.read_csv("Appeals_Courts.csv",sep=",")
Appeals = Appeals[[1,4,5]]
A = Appeals.shape[0]
Appeals_List = Appeals['PROV_CODE'].tolist()

In [3]:
#Create Dictionaries
Settlement_Dict = Settlements.set_index('OBJECTID').T.to_dict('list')
District_Dict = Districts.set_index('DIST_CODE').T.to_dict('list')
Appeals_Dict = Appeals.set_index('PROV_CODE').T.to_dict('list')

In [4]:
#Create Dictionaries for District Courthouse Distances

Dist_D = {}

R = 6371e3


for d in District_List:
    print(d)
    d_lon = District_Dict[d][0]
    d_lat = District_Dict[d][1]
    φ2 = np.radians(d_lat)


    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        φ1 = np.radians(s_lat)
        
        #Distance to District Court
        Δφ = np.radians(d_lat - s_lat)
        Δλ = np.radians(d_lon - s_lon)
        a = np.sin(Δφ/2) * np.sin(Δφ/2) + np.cos(φ1) * np.cos(φ2) * np.sin(Δλ/2) * np.sin(Δλ/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        Dist_D[s,d] = (R * c)/1000
        
    

0.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
201.0
202.0
203.0
204.0
205.0
206.0
207.0
301.0
302.0
303.0
304.0
305.0
306.0
307.0
308.0
309.0
310.0
401.0
402.0
403.0
404.0
405.0
406.0
407.0
408.0
409.0
501.0
502.0
503.0
504.0
505.0
506.0
507.0
601.0
602.0
603.0
604.0
605.0
606.0
607.0
608.0
609.0
610.0
611.0
612.0
613.0
614.0
615.0
616.0
617.0
618.0
619.0
620.0
621.0
622.0
701.0
702.0
703.0
704.0
705.0
801.0
802.0
803.0
804.0
805.0
806.0
807.0
901.0
902.0
903.0
904.0
905.0
906.0
907.0
908.0
909.0
910.0
911.0
912.0
913.0
914.0
915.0
1001.0
1002.0
1003.0
1004.0
1005.0
1006.0
1007.0
1101.0
1102.0
1103.0
1104.0
1105.0
1106.0
1107.0
1108.0
1109.0
1110.0
1111.0
1112.0
1113.0
1114.0
1115.0
1116.0
1117.0
1118.0
1119.0
1201.0
1202.0
1203.0
1204.0
1205.0
1206.0
1207.0
1208.0
1209.0
1210.0
1211.0
1301.0
1302.0
1303.0
1304.0
1305.0
1306.0
1307.0
1308.0
1309.0
1310.0
1311.0
1312.0
1313.0
1314.0
1315.0
1401.0
1402.0
1403.0
1404.0
1405.0
1406.0
1407.0
1

In [5]:
#Create Dictionaries for Appeals Courthouse Distances

Dist_A = {}

R = 6371e3

for a in Appeals_List:
    print(a)
    a_lon = Appeals_Dict[a][0]
    a_lat = Appeals_Dict[a][1]
    φ3 = np.radians(a_lat)

    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        φ1 = np.radians(s_lat)
        
        #Distance to District Court
        Δφ = np.radians(a_lat - s_lat)
        Δλ = np.radians(a_lon - s_lon)
        a1 = np.sin(Δφ/2) * np.sin(Δφ/2) + np.cos(φ1) * np.cos(φ3) * np.sin(Δλ/2) * np.sin(Δλ/2)
        c = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
        
        Dist_A[s,a] = (R * c)/1000
        
    

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0


In [6]:
#Calculate Min, Mean Distance to District Court 

min_d = min(Dist_D[d] for d in Dist_D)
avg_d = sum(Dist_D[d] for d in Dist_D) / len(Dist_D)

In [7]:
#Calculate Min, Mean Distance to Appeals Court
min_a =  min(Dist_A[a] for a in Dist_A)
avg_a = sum(Dist_A[a] for a in Dist_A) / len(Dist_A)

### Set Parameters

In [8]:
M = 10000
maxOpenD = 1000
maxOpenA = 100
maxDistance = 10000

#maxPopD- max capacity of each district court
#maxPopA - max capacity of each appeals court

#xS,yS 
#xD,yD
#xA,yA

#langS
#langD
#langA


#param d{i in Settlements,j in DistrictCourts} 
#	:= sqrt( (xS[i]-xD[j])^2 + (yS[i]-yD[j])^2); #distance between settlement and district court
#param a{i in Settlements,k in AppealsCourts} 
#	:= sqrt( (xA[k]-xS[i])^2 + (yA[k]-yS[i])^2); #distance between district court and appeals court	

### Create Model

In [9]:
#Create Model
m = Model("Afghanistan")

### Create Variables

##### d_ij

In [10]:
# Create d_i_j variables
d = {}
for i in Settlement_List:
    d[i] = {}
    for j in District_List:
        if Dist_D[i,j] < 150:
            d[i][j] = m.addVar(vtype=GRB.BINARY, name='d_%s_%s' % (i, j))
m.update()

In [11]:
# Create d_i_j transpose dictionary 
d_t = defaultdict(dict)
for key, val in d.items():
    for subkey, subval in val.items():
        d_t[subkey][key] = subval

##### a_ik

In [12]:
 # Create a_i_k variables
a = {}
for i in Settlement_List:
    a[i] = {}
    for k in Appeals_List:
        if Dist_A[i,k] < 350:
            a[i][k] = m.addVar(vtype=GRB.BINARY, name='a_%s_%s' % (i, k))
m.update()

In [13]:
# Create a_i_k transpose dictionary 
a_t = defaultdict(dict)
for key, val in a.items():
    for subkey, subval in val.items():
        a_t[subkey][key] = subval

##### c_jk

In [14]:
# Create c_j_k variables
c = {}
for j in District_List:
    c[j] = {}
    for k in Appeals_List:
        c[j][k] = m.addVar(vtype=GRB.BINARY, name='c_%s_%s' % (j, k))
m.update()

In [15]:
# Create c_j_k transpose dictionary 
c_t = defaultdict(dict)
for key, val in c.items():
    for subkey, subval in val.items():
        c_t[subkey][key] = subval

##### openD

In [16]:
#Create openD variables
openD = {}
for j in District_List:
    openD[j] = m.addVar(vtype=GRB.BINARY, name='openD_%s' % (j))
m.update()

##### openA

In [17]:
#Create openA variables
openA = {}
for k in Appeals_List:
    openA[k] = m.addVar(vtype=GRB.BINARY, name='openA_%s' % (k))
m.update()

### Create Constraints

##### One Courhouse Assignment Constraints

In [18]:
#One S -> D Assignment
for i in Settlement_List:
    m.addConstr(quicksum(d[i][j] for j in d[i]) == 1)
m.update()

In [19]:
#One S -> A Assignment
for i in Settlement_List:
    m.addConstr(quicksum(a[i][k] for k in a[i]) == 1)
m.update()

In [20]:
#One D -> A Assignment
for j in District_List:
    m.addConstr(quicksum(c[j][k] for k in c[j]) <= 1)
m.update()

In [21]:
# <gurobi.Model MIP instance Afghanistan: 90433 constrs, 6239918 vars, Parameter changes: LogFile=gurobi.log>

##### Max Open Courthouse Constraints

In [22]:
#Maximum District Courts Open
for j in District_List:
    m.addConstr(quicksum(d_t[j][i] for i in d_t[j]) <= M * openD[j])
    
m.addConstr(quicksum(openD[j] for j in District_List) <= maxOpenD)

m.update()

In [23]:
#Maximum Appeals Courts Open
for k in Appeals_List:
    m.addConstr(quicksum(a_t[k][i] for i in a_t[k]) <= M * openA[k])
    
m.addConstr(quicksum(openA[k] for k in Appeals_List) <= maxOpenA)

m.update()

In [24]:
m

<gurobi.Model MIP instance Afghanistan: 90869 constrs, 3186780 vars, Parameter changes: LogFile=gurobi.log>

In [ ]:
# <gurobi.Model MIP instance Afghanistan: 90869 constrs, 3,186,780 vars, Parameter changes: LogFile=gurobi.log>

##### Linking Constraint

### Set Objective Function

In [ ]:
 model.setObjective(
        quicksum(
            quicksum(
                Dist_D[i,j]*d[i,j] 
                for j in District_List) 
            for i in Settlement_List) + \
        quicksum(
            quicksum(
                Dist_A[i,k]*a[i,k] 
                for a in Appeals_List) 
            for i in Settlement_List), GRB.MINIMIZE)

In [ ]:


quicksum(
    quicksum(
        Dist_D[i,j]*d[i,j] 
        for j in District_List) 
    for i in Settlement_List) + \
quicksum(
    quicksum(
        Dist_A[i,k]*a[i,k] 
        for a in Appeals_List) 
    for i in Settlement_List)

### Optimize

In [ ]:
#NOTES - Look at poolsearch example, portfolio
#Make arrays of settlements, districts, courthouses
#d_ij = model.addVars(Settlements, Districts, vtype=GRB.BINARY, name='d_ij')
#
#

In [ ]:
S = 4
D = 2

In [ ]:
#Create d_ij
d_ij = [[0 for x in range(D)] for y in range(S)] 
for i in range(S):
    for j in range(D):
        d_ij[i][j] = m.addVar(vtype=GRB.BINARY, name="d_%s%s"%(i,j))

In [ ]:
#Create a_ik
Matrix[1][1] = 5

In [ ]:
#Create c_jk

In [ ]:
var = 3
print("I love %s"%var)

In [ ]:
d_ij